In [263]:
from dash import Dash, dash_table, dcc, callback, Output, Input, clientside_callback, _dash_renderer
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
from requests import get
from dash.dependencies import MATCH, ALL, State
from dash.exceptions import PreventUpdate
from dash import callback_context
from dash import html
import webbrowser
from threading import Timer
from dash_iconify import DashIconify


In [264]:
CSV_FILES = {
    "Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/CPS_Summary.csv",
    "Brands per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/BPCPS_Summary.csv",
    "Products per Brand": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPBPS_Summary.csv",
    "Products per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPCPS_Summary.csv"
}


In [265]:
type_map = {
        "Brands per Category": "brand",
        "Products per Category": "product",
        "Products per Brand": "product",
        "Category": "category"
    }

In [266]:
def load_data(radio_choice):
    col = type_map[radio_choice]
    df = pd.read_csv(
        CSV_FILES[radio_choice],
        usecols=[col, 'year', 'month','Popularity Score']
    )
    return df

In [267]:
def plot_time_series(filtered_df, radio_choice, selected_val, template):
    fig = px.line(filtered_df, x="date", y="Popularity Score",
                  template=template, title=f"{radio_choice} - {selected_val}")
    
    return fig

In [268]:
def plot_bar_chart(filtered_df, prod_names, selected_val, template):
    fig = px.bar(filtered_df, x="month_year", y='Popularity Score',
                 template=template, title=f"Top Products in {selected_val}:\n{', '.join(prod_names.astype(str))}")
    
    return fig

In [269]:
def get_template(theme):
    return "plotly_dark" if theme == "dark" else "plotly_white"

In [270]:
# df = pd.read_csv(CSV_FILES["Product per Category"])
# df.sort_values(by='PPCPS', inplace=True, ascending=False)
# # df.head()
# df.to_csv(CSV_FILES["Product per Category"], index=False)

In [271]:
theme_toggle = dmc.Switch(
    offLabel=DashIconify(icon="radix-icons:sun", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][8]),
    onLabel=DashIconify(icon="radix-icons:moon", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][6]),
    id="color-scheme-switch",
    persistence=True,
    color="grey",
)

In [272]:
def load_brand_data(radio_choice, select_cat, select_brand, template):
    # df = pd.read_csv(
    #     CSV_FILES[radio_choice],
    #     usecols=['brand', 'year', 'month', 'Popularity Score']
    # )
    cat_df = pd.read_csv(CSV_FILES["Category"])

    brand_df = load_data(radio_choice)

    cat_options = [{'label': val, 'value': val} for val in cat_df['category'].unique()]

    brand_df = brand_df[brand_df['category'] == select_cat].copy()

    brand_options = [{'label': val, 'value': val} for val in brand_df['brand'].unique()]

    if select_brand not in [opt["value"] for opt in brand_options]:
        return brand_options, "Please select a brand to view the time series data.", {'display': 'block'}
    
    filtered_df = brand_df[brand_df['brand'] == select_brand].copy()
    filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    
    return (
        cat_options, brand_options, 
        dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_brand, template)), 
        {'display': 'block'}
        )

In [273]:
def load_prod_data(radio_choice, selected_val, template):
    # df = pd.read_csv(
    #     CSV_FILES[radio_choice],
    #     usecols=['product', 'year', 'month', 'Popularity Score']
    # )

    df = load_data(radio_choice)

    filtered_df = df.head(6)
    # print(filtered_df.head())
    options = [{'label': str(y), 'value': str(y)} for y in sorted(filtered_df['year'].unique())]

    if selected_val not in [opt["value"] for opt in options]:
        return options, "Please select a product to view the populartiy  data.", {'display': 'none'}
    
    filtered_df = filtered_df[filtered_df['year'].astype(str) == selected_val].copy()
    prod_names = filtered_df['product']
    # print(prod_names)

    filtered_df['month_year'] = filtered_df['month'].astype(str) + "-" + filtered_df['year'].astype(str)

    return(
        options, 
        dcc.Graph(figure=plot_bar_chart(filtered_df, prod_names, selected_val, template)), 
        {'display': 'none'}
    )

In [274]:
def load_cat_data(radio_choice, selected_val, template):
    # df = pd.read_csv(
    #     CSV_FILES[radio_choice],
    #     usecols=['category', 'year', 'month', 'Popularity Score']
    # )

    df = load_data(radio_choice)

    options = [{'label': val, 'value': val} for val in sorted(df['category'].unique())]

    if selected_val not in [opt["value"] for opt in options]:
        return options, "Please select a category to view the time series data.", {'display': 'block'}
    
    filtered_df = df[df['category'] == selected_val].copy()
    filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    
    
    return (
        options, 
        dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, selected_val, template)), 
        {'display': 'block'}
    )

In [ ]:
app = Dash()

app.layout = dmc.MantineProvider(
    children=[
        theme_toggle,
        dcc.Store(id='theme-store'),
        dmc.Container([

            dmc.Title("Market Analysis Dashboard", 
                      style={"size":"h3", "textAlign": "center"}), 
        
            dmc.Grid([
                dmc.GridCol(
                    dmc.RadioGroup(
                    [dmc.Radio(i, value=i) for i in CSV_FILES.keys()],
                    id="radio-group",
                    value="Category",
                    size="md",
                    style={"marginBottom": "25px"}
                    ),
                    span=8
                ),

                dmc.GridCol(
                    dmc.Checkbox(
                    id="compare-checkbox",
                    labelPosition="right",
                    label="Compare",
                    color="#5c7cfa",
                    variant="filled",
                    size="sm",
                    radius="sm",
                    disabled=False,
                    indeterminate=False,
                    style={'display': 'block', 'justifyContent': 'center',
                        'margonBottom': '25px'},
                    ),
                    span=4,
                    style={"display": "flex", 
                           "alignItems": "top", 
                           "justifyContent": "flex-end"}
                ),
            ]),
            
            dmc.Select(
                id="dropdown",
                data=[],
                placeholder="Select",
                searchable=True,
                clearable=True,
                # Important for server-side search:
                # searchValue="",
                nothingFoundMessage="No matches found",
            ),

            dmc.Space(h=20),

            html.Div(id="plots")
        ], fluid=True,
    )],
)
@callback(
    Output("dropdown", "data"),
    Output("plots", "children"),
    Output("compare-checkbox", "style"),
    # Output('select-prod-year-dropdown', 'style'),
    # Output('select-prod-year-dropdown', 'data'),
    # Output("category-dropdown", "style"),
    Input("radio-group", "value"),
    Input("dropdown", "value"),
    # Input("category-dropdown", "searchValue"),
    Input("theme-store", "data"),
    # Input("compare-checkbox", "style"),
    # Input('select-prod-year-dropdown', 'value')
)
def combined_update(radio_choice, selected_val, theme):
    template = get_template(theme)

    col_map = {
        "Brands per Category": load_brand_data,
        "Products per Category": load_prod_data,
        "Products per Brand": load_prod_data,
        "Category": load_cat_data
    }

    if radio_choice in col_map:
        return col_map[radio_choice](radio_choice, selected_val, template)
    
    return [], "Invalid selection.", {'display': 'none'}
    
clientside_callback(
    """
    (switchOn) => {
       const theme = switchOn ? 'dark' : 'light';
        document.documentElement.setAttribute('data-mantine-color-scheme', theme);
        return theme;
    }
    """,
    Output("theme-store", "data"),
    Input("color-scheme-switch", "checked"),
)


# def open_browser():
#     webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == "__main__":
    # Timer(1, open_browser).start()  # Wait 1 second then open browser
    app.run(debug=True)

[2025-08-13 20:42:02,157] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\dash\_callback.py", line 688, in add_context
    raise err
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\dash\_callback.py", line 679, in add_context
    output_value = _invoke_callbac